# Imports

In [207]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import hyperspy.api as hs
from esmpy.estimators import SmoothNMF
import esmpy.datasets as ds

import matplotlib.pyplot as plt
import numpy as np

from esmpy.models.EDXS_function import print_concentrations_from_W

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generating artificial datasets and loading them

If the datasets were already generated, they are not generated again

**This is optional**

In [208]:
ds.generate_built_in_datasets(seeds_range=5)
spim = ds.load_particules(sample = 0)

# Loading your data and settings

There are three things to fill here : 
- An input filename in the ``hs.load()`` function
- The acquisition parameters in the ``spim.set_analysis_parameters()`` function. Be careful the previous metadata are overwritten. (I still need to build a function that does not overwrite the already present settings)
- if required you can set the concentrations of some elements in some phases with the function ``spim.set_fixed_W()``. The name of the phases can be changed.

In [231]:
spim = hs.load("../generated_datasets/FpBrgCaPv_N293_paper_/sample_5.hspy")

# spim.axes_manager[-1].offset = 12000
spim.set_signal_type("EDS_ESMPY")
spim.set_analysis_parameters(beam_energy = 200,
azimuth_angle = 0.0,
elevation_angle = 22.0,
tilt_stage = 0.0,
elements = ["Si","Mg","Fe"],
thickness = 200e-7,
density = 3.5,
detector_type = "SDD_efficiency.txt",
width_slope = 0.01,
width_intercept = 0.065,
xray_db = "default_xrays.json")
G = spim.build_G("bremsstrahlung", norm = True)
fW = spim.set_fixed_W({"p0" : {"Si" : 0.0},"p1" : {"Fe" : 0.0}, "p2" : {"Mg" : 0.0}})

# Problem solving

Full hyperspy syntax

## Loading analysis parameters

In [232]:
G = spim.build_G("bremsstrahlung", norm = True)
shape_2d = spim.shape_2d
phases = spim.phases
maps = spim.maps
print(G)
est = SmoothNMF( n_components = 3,tol=0.000001, max_iter = 1000, G = None, mu=0, lambda_L=0,  force_simplex=False) #  true_D = phases, true_A = weights

<bound method EDS_ESMPY.update_G of <EDS_ESMPY, title: , dimensions: (64, 64|500)>>


## Calculating the decomposition

/!\ Depending on the parameters you choose and the size of the data it might take a while

In [233]:
out = spim.decomposition(algorithm = est, return_info=True)

TypeError: To perform a decomposition the data must be of the float or complex type, but the current type is 'int64'. To fix this issue, you can change the type using the change_dtype method (e.g. s.change_dtype('float64')) and then repeat the decomposition.
No decomposition was performed.

## Getting the losses and the results of the decomposition

- First cell : Printing the resulting concentrations.
- Second cell : Ploting the resulting spectra
- Thrid cell : Ploting the resulting abundances

In [234]:
print_concentrations_from_W(est.W_, elements = spim.metadata.Sample.elements)

AttributeError: 'SmoothNMF' object has no attribute 'W_'

In [235]:
spim.plot_decomposition_loadings(3)

RuntimeError: No learning results found. A 'decomposition' needs to be performed first.

In [236]:
spim.plot_decomposition_factors(3)

RuntimeError: No learning results found. A 'decomposition' needs to be performed first.

# Problem Solving

With the usual scikit use

In [249]:
G = spim.build_G("bremsstrahlung")
shape_2d = spim.shape_2d
# phases, weights = spim.phases, spim.weights
X = spim.X
est = SmoothNMF( n_components = 3,tol=0.000001, max_iter = 1000, G = None, lambda_L= 0, mu=0, force_simplex=False)

In [250]:
pixels_side = 64

Ximg = X.reshape((-1, pixels_side,pixels_side))
print(Ximg.shape)

(500, 64, 64)


In [251]:
downsampling_factors = [8,4,2,1]
for i in range(len(downsampling_factors)):
    n = downsampling_factors[i]
    X_ = downsample(Ximg,n)
    Xflat = X_.reshape((X_.shape[0], -1))
    print(Xflat.shape)
    D = est.fit_transform(Xflat)
    Hflat = est.H_
    W = est.W_
 

    plot_results(spim.phases, W, downsample_flat(spim.maps,n), Hflat, pixels_side//n)

(500, 64)
3
It 10 / 1000: loss 0.008,  152.230 it/s
It 20 / 1000: loss 0.008,  195.921 it/s
It 30 / 1000: loss 0.008,  222.997 it/s
It 40 / 1000: loss 0.008,  241.488 it/s


/Users/maxmartinezruts/.local/share/virtualenvs/esmpy-kBfF6zeU/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


It 50 / 1000: loss 0.008,  250.647 it/s
It 60 / 1000: loss 0.008,  259.340 it/s
It 70 / 1000: loss 0.008,  257.538 it/s
It 80 / 1000: loss 0.008,  250.887 it/s
It 90 / 1000: loss 0.008,  254.733 it/s
It 100 / 1000: loss 0.008,  262.358 it/s
It 110 / 1000: loss 0.008,  272.452 it/s
It 120 / 1000: loss 0.008,  278.930 it/s
It 130 / 1000: loss 0.008,  282.475 it/s
It 140 / 1000: loss 0.008,  288.089 it/s
It 150 / 1000: loss 0.008,  293.950 it/s
It 160 / 1000: loss 0.008,  298.908 it/s
It 170 / 1000: loss 0.008,  303.778 it/s
It 180 / 1000: loss 0.008,  307.921 it/s
It 190 / 1000: loss 0.008,  311.957 it/s
It 200 / 1000: loss 0.008,  314.019 it/s
It 210 / 1000: loss 0.008,  316.136 it/s
It 220 / 1000: loss 0.008,  319.324 it/s
It 230 / 1000: loss 0.008,  320.259 it/s
It 240 / 1000: loss 0.008,  323.644 it/s
It 250 / 1000: loss 0.008,  325.682 it/s
It 260 / 1000: loss 0.008,  328.457 it/s
It 270 / 1000: loss 0.008,  332.663 it/s
It 280 / 1000: loss 0.008,  332.399 it/s
It 290 / 1000: loss 0

TypeError: downsample() missing 1 required positional argument: 'n'

In [244]:
def downsample(X,n):
    b = X.shape[2]//n
    return X.reshape(X.shape[0], -1, n, b, n).sum((-1, -3)) /(n*n)

def downsample_flat(X,n):
    X = X.reshape((-1, pixels_side,pixels_side))
    X = downsample(X)
    return X.reshape((-1, pixels_side*pixels_side//n//n))


In [245]:
from esmpy.measures import find_min_angle, find_min_MSE, ordered_mse

def plot_results(Ddot, D, Hdotflat, Hflat, n_pixel_side):
    fontsize = 30
    scale = 15
    aspect_ratio = 1.4
    marker_list = ["-o","-s","->","-<","-^","-v","-d"]
    mark_space = 20
    # cmap = plt.cm.hot_r    
    cmap = plt.cm.gray_r
    vmax = 1
    vmin = 0
    K = len(H)
    L = len(D)
    
    angles, true_inds = find_min_angle(Ddot.T, D.T, unique=True, get_ind=True)
    mse = ordered_mse(Hdotflat, Hflat, true_inds)


    fig, axes = plt.subplots(K,3,figsize = (scale/K * 3 * aspect_ratio,scale))
    x = np.linspace(0,1, num = L)
    for i in range(K): 
        axes[2,i].plot(x,Ddot.T[i,:],'bo',label='truth',linewidth=4)
        axes[2,i].plot(x,D[:,true_inds[i]],'r-',label='reconstructed',markersize=3.5)
        axes[2,i].set_title("{:.2f} deg".format(angles[i]),fontsize = fontsize-2)
        axes[2,i].set_xlim(0,1)

        axes[1,i].imshow((Hflat[true_inds[i],:]).reshape(n_pixel_side,n_pixel_side),vmin = vmin, vmax = vmax , cmap=cmap)
        axes[1,i].set_title("{:.2f} MSE".format(mse[true_inds[i]]),fontsize = fontsize-2)
        # axes[i,1].set_ylim(0.0,1.0)
        axes[1,i].tick_params(axis = "both",labelleft = False, labelbottom = False,left = False, bottom = False)

        im = axes[0,i].imshow(Hdotflat[i].reshape(n_pixel_side,n_pixel_side),vmin = vmin, vmax = vmax, cmap=cmap)
        axes[0,i].set_title("Phase {}".format(i),fontsize = fontsize)
        axes[0,i].tick_params(axis = "both",labelleft = False, labelbottom = False,left = False, bottom = False)
        axes[2,0].legend()

    rows = ["True maps","Reconstructed maps","Spectra"]

    for ax, row in zip(axes[:,0], rows):
        ax.set_ylabel(row, rotation=90, fontsize=fontsize)


    fig.subplots_adjust(right=0.84)
    # put colorbar at desire position
    cbar_ax = fig.add_axes([0.85, 0.5, 0.01, 0.3])
    fig.colorbar(im,cax=cbar_ax)

    # fig.tight_layout()

    plt.show()